In [7]:
import pygame
import random
from typing import List, Tuple

GRID_SIZE = 10           # ukuran grid: GRID_SIZE x GRID_SIZE
CELL_SIZE = 64          
MARGIN = 2               
NUM_COINS = 15           # jumlah koin
FPS = 60

# Warna (R, G, B)
BG = (20, 20, 24)
GRID_LINE = (60, 60, 70)
COIN_COLOR = (240, 210, 60)
HUMAN_COLOR = (90, 200, 250)
AI_COLOR = (255, 100, 120)
TEXT = (235, 235, 240)
PANEL_BG = (30, 30, 38)

WIDTH = GRID_SIZE * CELL_SIZE + 80
HEIGHT = GRID_SIZE * CELL_SIZE + 80  

# Posisi awal
HUMAN_START = (0, GRID_SIZE - 1)          # kiri-bawah
AI_START = (GRID_SIZE - 1, 0)             # kanan-atas

Vec2 = Tuple[int, int]

def manhattan(a: Vec2, b: Vec2) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def grid_to_px(cell: Vec2) -> Tuple[int, int]:
    x, y = cell
    return x * CELL_SIZE, y * CELL_SIZE

def clamp(n, lo, hi):
    return max(lo, min(hi, n))

def nearest_coin(from_pos: Vec2, coins: List[Vec2]) -> Vec2 | None:
    if not coins:
        return None
    # pilih koin dengan jarak Manhattan terkecil
    best = min(coins, key=lambda c: manhattan(from_pos, c))
    return best

def step_towards(src: Vec2, dst: Vec2) -> Vec2:
    # Melangkah 1 sel (tidak bisa langsung diagonal) 
    x, y = src
    dx = dst[0] - x
    dy = dst[1] - y
    if abs(dx) >= abs(dy):
        x += 1 if dx > 0 else (-1 if dx < 0 else 0)
    else:
        y += 1 if dy > 0 else (-1 if dy < 0 else 0)
    # pastikan tetap di grid
    x = clamp(x, 0, GRID_SIZE - 1)
    y = clamp(y, 0, GRID_SIZE - 1)
    return (x, y)

def is_adjacent(a: Vec2, b: Vec2) -> bool:
    return manhattan(a, b) == 1

class Game:
    def __init__(self):
        self.reset()

    def reset(self):
        self.human = HUMAN_START
        self.ai = AI_START
        # generate posisi koin unik
        taken = {self.human, self.ai}
        coins = set()
        while len(coins) < NUM_COINS:
            c = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
            if c not in taken and c not in coins:
                coins.add(c)
        self.coins = sorted(list(coins))
        self.human_score = 0
        self.ai_score = 0
        self.turn = "HUMAN"  # "HUMAN" atau "AI"
        self.game_over = False
        self.message = "Your turn"

    def collect_coin_if_any(self, who: str):
        pos = self.human if who == "HUMAN" else self.ai
        if pos in self.coins:
            self.coins.remove(pos)
            if who == "HUMAN":
                self.human_score += 1
            else:
                self.ai_score += 1

    def update_game_over(self):
        if not self.coins:
            self.game_over = True
            if self.human_score > self.ai_score:
                self.message = "Game Over — You win! 🎉"
            elif self.ai_score > self.human_score:
                self.message = "Game Over — AI wins! 🤖"
            else:
                self.message = "Game Over — Draw! 🤝"

    def human_try_move(self, target_cell: Vec2):
        if self.game_over or self.turn != "HUMAN":
            return
        if is_adjacent(self.human, target_cell):
            self.human = target_cell
            # ambil koin jika ada
            self.collect_coin_if_any("HUMAN")
            # cek selesai
            self.update_game_over()
            if not self.game_over:
                self.turn = "AI"
                self.message = "AI thinking…"
        else:
            pass

    def ai_move(self):
        if self.game_over or self.turn != "AI":
            return
        if not self.coins:
            self.update_game_over()
            return
        target = nearest_coin(self.ai, self.coins)
        if target is None:
            self.update_game_over()
            return
        self.ai = step_towards(self.ai, target)
        self.collect_coin_if_any("AI")
        self.update_game_over()
        if not self.game_over:
            self.turn = "HUMAN"
            self.message = "Your turn"

def draw_grid(surface):
    # grid background
    surface.fill(BG)
    # cells border
    for x in range(GRID_SIZE + 1):
        px = x * CELL_SIZE
        pygame.draw.line(surface, GRID_LINE, (px, 0), (px, GRID_SIZE * CELL_SIZE), 1)
    for y in range(GRID_SIZE + 1):
        py = y * CELL_SIZE
        pygame.draw.line(surface, GRID_LINE, (0, py), (GRID_SIZE * CELL_SIZE, py), 1)

def draw_coins(surface, coins: List[Vec2]):
    for c in coins:
        px, py = grid_to_px(c)
        cx = px + CELL_SIZE // 2
        cy = py + CELL_SIZE // 2
        r = CELL_SIZE // 4
        pygame.draw.circle(surface, COIN_COLOR, (cx, cy), r)

def draw_human(surface, pos: Vec2):
    px, py = grid_to_px(pos)
    pad = CELL_SIZE // 8
    rect = pygame.Rect(px + pad, py + pad, CELL_SIZE - 2 * pad, CELL_SIZE - 2 * pad)
    pygame.draw.rect(surface, HUMAN_COLOR, rect, border_radius=8)

def draw_ai(surface, pos: Vec2):
    px, py = grid_to_px(pos)
    pad = CELL_SIZE // 8
    p1 = (px + CELL_SIZE // 2, py + pad)
    p2 = (px + pad, py + CELL_SIZE - pad)
    p3 = (px + CELL_SIZE - pad, py + CELL_SIZE - pad)
    pygame.draw.polygon(surface, AI_COLOR, [p1, p2, p3])

def draw_panel(surface, font, small_font, game: Game):
    panel_rect = pygame.Rect(0, GRID_SIZE * CELL_SIZE, WIDTH, 80)
    pygame.draw.rect(surface, PANEL_BG, panel_rect)

    text1 = font.render(
        f"Coins left: {len(game.coins)}   Score — You: {game.human_score} | AI: {game.ai_score}",
        True, TEXT
    )
    surface.blit(text1, (16, GRID_SIZE * CELL_SIZE + 10))

    turn_txt = ("Turn: YOU" if game.turn == "HUMAN" else "Turn: AI") if not game.game_over else "Turn: —"
    text2 = small_font.render(
        f"{turn_txt}    Tip: Click an adjacent cell to move.  Press R to restart.",
        True, TEXT
    )
    surface.blit(text2, (16, GRID_SIZE * CELL_SIZE + 44))

    if game.message:
        msg = small_font.render(game.message, True, TEXT)
        surface.blit(msg, (WIDTH - msg.get_width() - 16, GRID_SIZE * CELL_SIZE + 10))

def cell_from_mouse(mx: int, my: int) -> Vec2 | None:
    if my >= GRID_SIZE * CELL_SIZE:
        return None
    gx = mx // CELL_SIZE
    gy = my // CELL_SIZE
    return (int(gx), int(gy))

def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("=== COIN HUNT ===")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("consolas", 22)
    small_font = pygame.font.SysFont("consolas", 18)

    game = Game()

    running = True
    ai_timer = 0
    AI_DELAY_MS = 400  

    while running:
        dt = clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    game.reset()

            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                cell = cell_from_mouse(*event.pos)
                if cell is not None:
                    game.human_try_move(cell)
                    if game.turn == "AI" and not game.game_over:
                        ai_timer = AI_DELAY_MS 

        # AI move
        if game.turn == "AI" and not game.game_over:
            ai_timer -= dt
            if ai_timer <= 0:
                game.ai_move()

        # render
        screen.fill(BG)
        draw_grid(screen)
        draw_coins(screen, game.coins)
        draw_human(screen, game.human)
        draw_ai(screen, game.ai)
        draw_panel(screen, font, small_font, game)
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()
